## Accessing Cloud Optimized Raster Data

With the AWS STAC API (TODO update official name) we can search and load various geospatial datasets. Below we'll work with the Copernicus GLO-30 Digital Elevation Model, sourced from Cloud Optimized GeoTIFFs hosted on AWS. In this example, we read in elevation data using a bounding box.

First we install any necessary packages. Please note that the following block of code only needs to be run if needed within a new workspace and that you may need to restart the kernel to use updated packages. The code block only needs to be run once for each AWS Sagemaker Studio profile.

In [ ]:
%pip install --upgrade pip -q
%pip install matplotlib pystac_client stackstac rasterio folium -q

We'll use pystac_client to submit our search query, stackstac to load and composite our rasters as an Xarray DataArray, and rio-tiler to performantally load individual Cloud Optimized GeoTIFFs.

### Querying the AWS STAC API with Pystac Client

First, we define the URL to our STAC API, a service that allows us to submit search queries for spatio-temporal assets (like elevation raster tiles).

Next, we open this URL with pystac_client, which allows us to use the client's search method. `client.search` accepts many different arguments to specify the search query. Common query parameters are `time_range` and `bbox` (the spatial bounding box). Since the DEM was collected at one point in time, we will only specify the "where" with a `bbox` describing the minx, miny, maxx, maxy coordinates in lat/lon degrees.

In [ ]:
import os
from pystac_client import Client
import stackstac
# making a connection to AWS STAC API
API_ROOT_URL = "https://1hfhn8bf6g.execute-api.us-west-2.amazonaws.com"
client = Client.open(API_ROOT_URL)

# our search requirements, a bounding box for San Ignacio. Belize
bbox = [-89.19605011359278, 17.028707271735962, -88.94369134663197, 17.263285211322446]
source_search = client.search(
        collections=["cop-dem-glo-30"], bbox=bbox
    )
# getting the metadata and links to data assets in STAC format
source_items = source_search.get_all_items()
print("Number of elevation tiles: ", len(source_items))

### Loading an OSM descriptive map of the same area

To better understand our area of interest, we can create a descriptive map with our bounding box. We'll load an OpenStreetMap basemap that includes place names, annotations for water and forested areas, and roads. We'll use `folium` to access and plot this map. To learn more about `folium`, check out the [documentation](https://python-visualization.github.io/folium/). We'll use our bounding box to specify where we want to center our map and pick a zoom level that shows us detail around San Ignacio.

In [ ]:
center_point = ((bbox[1] + bbox[3])/2, (bbox[0] + bbox[2])/2)
center_point

In [ ]:
import folium
m = folium.Map(location=center_point, zoom_start=13)


### Loading DEM raster into an Xarray DataArray

There are multiple options for loading STAC items into in-memory arrays. `stackstac` allows you to specify a list of items from a `pystac_client` search and load those items as a single xarray DataArray. Similar to it's name, `stackstac` handles loading and then stacking individual COGs that fall within our `pystac` query. Since our `stack` DataArray has two tiles, we need to composite the tiles within the bounding box into one raster.

In [ ]:
# converting STAC items into an xarray data array
stack = stackstac.stack(source_items, epsg = 4326)

print(type(stack))

stack

We'll load this into memory so that later operations that use this array are faster.

In [ ]:
import rasterio
from dask.diagnostics import ProgressBar

with ProgressBar() and rasterio.Env(AWS_NO_SIGN_REQUEST=True):
    stack = stack.compute()

Before compositing, let's first plot the tiles individually tile to see if there are any nodata values we need to handle. A DataArray's plot method gives us a quick look at the raster.

In [ ]:
import matplotlib
stack.isel(time=0).plot(cmap="viridis", robust=True)

In [ ]:
stack.isel(time=1).plot(cmap="viridis", robust=True)

It looks like 0 is the nodata value, which is evident by the hard boundary between 0 valued pixels and data pixels. We can mask these nodata areas with the `where` method so that they don't affect the composite calculation.

In [ ]:
stack = stack.where(stack!=0)

In [ ]:
stack.isel(time=1).plot(cmap="viridis", robust=True)

In [ ]:
stack.isel(time=0).plot(cmap="viridis", robust=True)

Below we compute a mean composite between the two tiles.

In [ ]:
# taking the mean along the time dimension to combine the two elevation tiles
composite = stack.mean(dim="time")
composite

In [ ]:
composite.plot(cmap="viridis", robust=True)

In the above plot, we can see the elevation contour of the Mopan river, which flows near San Ignacio from the north to the southwest corner of our bounding box.